In [1]:
import pandas as pd
import altair as alt
from datetime import datetime
from pprint import pprint

In [2]:
# https://vita.had.co.nz/papers/tidy-data.pdf

cc_data = pd.read_csv("datasets/cc_data.csv")
cc_data["timestamp"] = pd.to_datetime(cc_data["timestamp"])
lt_data = pd.read_csv("datasets/loyalty_data.csv")
lt_data["timestamp"] = pd.to_datetime(lt_data["timestamp"])

In [3]:
cc_data.columns, lt_data.columns

(Index(['timestamp', 'location', 'price', 'last4ccnum'], dtype='object'),
 Index(['timestamp', 'location', 'price', 'loyaltynum'], dtype='object'))

In [4]:
cc_num, lt_num = list(cc_data["last4ccnum"]), list(lt_data["loyaltynum"])
cc_timestamp, lt_timestamp = list(cc_data["timestamp"].dt.date), list(lt_data["timestamp"].dt.date)
cc_location, lt_location = list(cc_data["location"]), list(lt_data["location"])
cc_price, lt_price = list(cc_data["price"]), list(lt_data["price"])

In [5]:
keys_num_cc = {}
keys_num_lt = {}
keys_cc = []
keys_lt = []
for c in range(len(cc_timestamp)):
    for l in range(len(lt_timestamp)):
        if (cc_timestamp[c] == lt_timestamp[l]) and (cc_location[c] == lt_location[l]) and (cc_price[c] == lt_price[l]):
            keys_cc.append(c)
            keys_lt.append(l)
            keys_num_cc[str(cc_num[c])] = lt_num[l]
            keys_num_lt[str(lt_num[l])] = cc_num[c]

In [6]:
cc_copy = cc_data.copy()
lt_copy = lt_data.copy()

In [7]:
cc_dropped = cc_copy.drop(keys_cc)
not_keys_cc = list(cc_dropped.reset_index()["index"])
lt_dropped = lt_copy.drop(keys_lt)
not_keys_lt = list(lt_dropped.reset_index()["index"])

cc_copy.loc[keys_cc,"link"] = "both"
cc_copy.loc[not_keys_cc,"link"] = "credit"

lt_copy.loc[not_keys_lt, "link"] = "loyalty"
cc_copy["loyaltynum"] = ""
cc_copy["loyaltynum"] = cc_copy["last4ccnum"].map(lambda x : keys_num_cc[str(x)])

cc_copy = cc_copy.append(lt_copy.iloc[not_keys_lt])
cc_copy = cc_copy.reset_index()

cc_copy["last4ccnum"] = cc_copy["loyaltynum"].map(lambda x : keys_num_lt[str(x)])

In [8]:
cc_copy = cc_copy.drop(columns=["index"])

In [9]:
cc_copy.to_csv("datasets/linked.csv", index=False)